In [1]:
import tkinter as tk
from tkinter import scrolledtext
import re
from nltk.stem import WordNetLemmatizer

# Reading Text File
w = []
with open('final.txt', 'r', encoding="utf8") as f:
    file_name_data = f.read()
    file_name_data = file_name_data.lower()
    w = re.findall('\w+', file_name_data)

# Counting Word Frequency
def counting_words(words):
    word_count = {}
    for word in words:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

# Calculating Word Probabilities
def prob_cal(word_count_dict, total_words):
    probs = {}
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key] / total_words
    return probs

# LemmWord: extracting and adding root word i.e. Lemma using WordNetLemmatizer from NLTK
def LemmWord(word):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)

# Deleting letters from the words
def DeleteLetter(word):
    delete_list = []
    split_list = []

    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))

    for a, b in split_list:
        delete_list.append(a + b[1:])
    return delete_list

# Switching two letters in a word
def Switch_(word):
    split_list = []
    switch_l = []

    for i in range(len(word)):
        split_list.append((word[0:i], word[i:]))

    switch_l = [a + b[1] + b[0] + b[2:] for a, b in split_list if len(b) >= 2]
    return switch_l

# Replacing the letter one-by-one from the list of alphs
def Replace_(word):
    split_l = []
    replace_list = []

    for i in range(len(word)):
        split_l.append((word[0:i], word[i:]))
    alphs = 'abcdefghijklmnopqrstuvwxyz'
    replace_list = [a + l + (b[1:] if len(b) > 1 else '') for a, b in split_l if b for l in alphs]
    return replace_list

# Making pairs of the split words and storing new words in a list with one new character at each location
def insert_(word):
    split_l = []
    insert_list = []

    for i in range(len(word) + 1):
        split_l.append((word[0:i], word[i:]))

    alphs = 'abcdefghijklmnopqrstuvwxyz'
    insert_list = [a + l + b for a, b in split_l for l in alphs]
    return insert_list

# Collecting all the words in a set(so that no word will repeat)
def colab_1(word, allow_switches=True):
    colab_1 = set()
    colab_1.update(DeleteLetter(word))
    if allow_switches:
        colab_1.update(Switch_(word))
    colab_1.update(Replace_(word))
    colab_1.update(insert_(word))
    return colab_1

# Collecting words using by allowing switches
def colab_2(word, allow_switches=True):
    colab_2 = set()
    edit_one = colab_1(word, allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = colab_1(w, allow_switches=allow_switches)
            colab_2.update(edit_two)
    return colab_2

# Only storing those values which are in the vocab
def get_corrections(word, probs, vocab, n=2):
    suggested_word = []
    best_suggestion = []
    suggested_word = list(
        (word in vocab and word) or colab_1(word).intersection(vocab)
        or colab_2(word).intersection(vocab))

    best_suggestion = [[s, probs[s]] for s in list(reversed(suggested_word))]
    return best_suggestion

# Vocabulary
main_set = set(w)

# Counting word function
word_count = counting_words(main_set)

# Calculating probability
probs = prob_cal(word_count, len(w))

# Function to perform spell checking and display suggestions in the GUI
def check_spelling():
    input_word = entry.get().lower()

    # Only storing correct words
    corrections = get_corrections(input_word, probs, main_set, 2)

    # Display suggestions in the text area
    result_text.delete(1.0, tk.END)  # Clear previous results
    for i, (word, probability) in enumerate(corrections):
        if i < 3:
            result_text.insert(tk.END, f"{word} \n")
        else:
            break

# Create the main window
root = tk.Tk()
root.title("Spell Checker")

# Styling the GUI
root.geometry("500x400")
root.configure(bg="#f0f0f0")

# Create and place GUI components
label = tk.Label(root, text="Spell Checker", font=("Helvetica", 16), bg="#f0f0f0")
label.pack(pady=10)

entry = tk.Entry(root, width=30, font=("Helvetica", 12))
entry.pack(pady=10)

check_button = tk.Button(root, text="Check Spelling", command=check_spelling, font=("Helvetica", 12), bg="#4CAF50", fg="white")
check_button.pack(pady=10)

result_text = scrolledtext.ScrolledText(root, width=40, height=10, wrap=tk.WORD, font=("Helvetica", 10))
result_text.pack(pady=10)

# Run the GUI
root.mainloop()
